# Обучение модели

In [5]:
import pandas as pd
import numpy as np
import gdown 
import seaborn as sns
import scipy.sparse as sps
import pickle
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.sparse import hstack, vstack 

## Загрузка данных

In [2]:
# Загрузка матрицы с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1D2WB0E7FEHrYHpiXMYezo_Gx8UFGg0_H",  
               output="rating_matrix.npz",
               quiet=False)
# Импорт матрицы
rating_matrix = sps.load_npz("rating_matrix.npz")

Downloading...
From: https://drive.google.com/uc?export=download&id=1D2WB0E7FEHrYHpiXMYezo_Gx8UFGg0_H
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\rating_matrix.npz
100%|██████████| 19.5M/19.5M [00:01<00:00, 10.6MB/s]


In [3]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\test_transacrion_df.csv
100%|██████████| 52.4M/52.4M [00:04<00:00, 11.4MB/s]


In [9]:
# Загрузка кодировщика user_id
gdown.download(url="https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh",  
               output="encoder.pkl",
               quiet=False)
# Загрузка кодировщика
with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)

Downloading...
From: https://drive.google.com/uc?export=download&id=1eodl9OlaYy3NTu9TpbtPLHNurXxeHlhh
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\encoder.pkl
100%|██████████| 800k/800k [00:00<00:00, 3.32MB/s]
c:\code\rec_sys\hse_mlds_recsys_project\.conda\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.4.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
# Загрузка датафрейма с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c",  
               output="ratings.csv",
               quiet=False)
rating_df = pd.read_csv("ratings.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\ratings.csv
100%|██████████| 211M/211M [00:18<00:00, 11.5MB/s] 


In [11]:
rating_matrix

<9137821x9137821 sparse matrix of type '<class 'numpy.int64'>'
	with 9137821 stored elements in Compressed Sparse Row format>

In [12]:
# Целесообразно брать только непустые строки и столбцы 
cnt_users = rating_df['user_id'].nunique()
cnt_items = rating_df['product_id'].nunique()
# Ограничение матрицы с рейтингами
rating_matrix = rating_matrix[:cnt_users, :cnt_items+1]
rating_matrix

<100000x49414 sparse matrix of type '<class 'numpy.int64'>'
	with 9081563 stored elements in Compressed Sparse Row format>

# Метрика качества

In [24]:
def precision_at_k(relevant, predicted, k: int = 10):
    """ 
        Функиця расчета Precision@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    return len(set(relevant[:k]) & set(predicted[:k]))/k 

def rel_item(relevant, predicted):
    """
        Функция рассчитывает количество релевантных Item
    """
    result = [0]*max(len(relevant), len(predicted))
    items = min(len(relevant), len(predicted))
    for i in range(items):
        result[i] = int(relevant[i] == predicted[i])
    return result  


def ap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    iter_cnt = min(len(relevant), k)
    for i in range(1, iter_cnt+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def map_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_apk = 0
    for user in range(users):
        sum_apk += ap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_apk/users

def nap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета normalize AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    k = min(len(relevant), k)
    for i in range(1, k+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def mnap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_napk = 0
    for user in range(users):
        sum_napk += nap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_napk/users

def hitrate_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета Hitrate@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    
    cnt_user = len(predicted) # Количество пользователей    
    cnt_valid_user = 0
    for user in range(cnt_user):
        cnt_valid_user += int(len(set(relevant[user][:k]) & set(predicted[user][:k])) > 0) 

    return cnt_valid_user/cnt_user

def ndsg_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета nDSG@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    idsg_at_k = sum([1/np.log2(k+1) for k in range(1, k+1)])
    k = min(len(relevant), k)
    dsg_at_k = 0
    for k in range(1, k+1):
        dsg_at_k += int(relevant[k-1] == predicted[k-1])/np.log2(k+1)
    return dsg_at_k/idsg_at_k




## Коллаборативная фильтрация (User2User)


В качестве функции схожести будет использовано две метрики:

1. Корреляция Пирсона $$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$

2. Мера Жаккара

$$ s(u, v) = \frac{|I_u \cap I_v|}{|I_u \cup I_v|} $$


Корреляция Пирсона немного видоизменена, чтобы подходить под текущую задачу.</br> 
квадрат в формуле можно опустить т.к. рейтинг принимает два значения 0 или 1 (квадрат которых 0 или 1)

Во всех формулах 
* $I_u$ - множество продуктов, купленных пользователем $u$.
* $r_{ui}$ - покупал ли пользователь $u$ продукт $i$ (0 или 1).

Множество соседей определим как $$N(u) = \{ v \in U \setminus \{u\} \mid s(u, v) > \alpha\},$$ где $\alpha \, - $ гиперпараметр.



Для агрегации используется следующая формула:
$$
\hat{r}_{ui} = \frac{\sum_{v \in N(u)} s(u, v) r_{vi}}{\sum_{v \in N(u)} |s(u, v)|}
$$

In [14]:
def pearson(rating_matrix, user_vector):
    """
        Функция расчета корреляции Пирсона
    """
    # Числитель
    numerator = rating_matrix.dot(user_vector.T)
    n = rating_matrix.shape[1]
    # Знаменатель
    denominator = np.sqrt(user_vector.dot(np.ones(shape=(n, 1)))) * np.sqrt(rating_matrix.dot(np.ones(shape=(n, 1))))
    ans = numerator / denominator
    return ans.toarray()

In [15]:
# def get_union(rating_matrix, uid: int = 0):
#     unions = []
#     for i in range(rating_matrix.shape[0]):
#         ans = rating_matrix[i] + rating_matrix[uid]
#         union = sum(np.where(ans.toarray()[0] >= 1, 1, 0))
#         unions.append(union)
#     return np.array(unions)


# def jaccard(ratings_matrix, uid: int):
#     # Пересечение 
#     inter = rating_matrix.dot(rating_matrix[uid].T)
#     union = get_union(rating_matrix=rating_matrix, uid=0)
    
#     return inter / union 

In [16]:
class User2User():

    def __init__(self, rating_matrix, similarity_func, encoder):
        assert similarity_func in [pearson]
        self.rating_matrix = rating_matrix
        self.similarity_func = similarity_func
        self.alpha = 0.02
        self.encoder = encoder

    def predict(self, users_to_recommend: list, n_jobs: int = 3, k: int = 10):
        """
            Параллельное вычисление рекомендаций для пользователей
        """
        predictions = Parallel(n_jobs=3)(delayed(self.recommend)(uid=uid, k=k) for uid in users_to_recommend)
        return predictions

    def cold_start(self):
        """
            Функция холодного старта
            возвращает популярные продукты по убываюнию
        """
        return np.argsort(rating_matrix.T.dot(np.ones(shape=(rating_matrix.shape[1], 1))).T)[::-1]

    def recommend(self,uid: int, k: int):
        """
            Расчет рекомендаций
        """

        # Если ранее такого пользователя не было, то применяется холодный старт
        try:
            uid = self.encoder.transform(np.array([uid]))
        except:
            return self.cold_start()

        # Вектор пользователя
        user_vector = self.rating_matrix[uid]
        # Расчет схожести пользователя с остальными пользователями
        sim_vector = self.similarity_func(rating_matrix=self.rating_matrix, user_vector=user_vector)
        # Индексы со значением функции схожести выше порогового значения
        collab_ids = np.where(sim_vector >= self.alpha)[0]
        # Исключается индекс самого объекта
        collab_ids = collab_ids[collab_ids != uid]
        # Расчет рекомендации 
        sum_of_suvs = np.sum(sim_vector[collab_ids])
        suv_matrix = (self.rating_matrix[collab_ids, ].T.dot(sim_vector[collab_ids])).T/sum_of_suvs
        # rui_est = np.sum(suv_matrix, axis=0)

        return np.argsort(suv_matrix[0])[::-1][:k]

In [17]:
model = User2User(rating_matrix=rating_matrix, 
                  similarity_func=pearson, 
                  encoder=encoder)

In [21]:
# Пользователи с их релевантными покупками
relevant = (test_df.sort_values(by=["user_id", "add_to_cart_order"])
            .groupby(['user_id'])
            .agg({'product_id': 'unique'})['product_id'])

In [22]:
%%time
# Рекомендации
predict = model.predict(relevant.index, n_jobs=3, k=10)

CPU times: total: 20min 57s
Wall time: 3h 12min 34s


Оценки качества модели

In [32]:
metrics = {}
relevant = relevant.to_list()
metrics["MAP@k"] = map_at_k(relevant=relevant, predicted=predict, k=10)
metrics["MNAP@k"] = mnap_at_k(relevant=relevant, predicted=predict, k=10)
metrics["Hitrate@k"] = hitrate_at_k(relevant=relevant, predicted=predict, k=10)
sum_ndsg = 0
sum_precision = 0
for user in range(len(relevant)):
    rel = relevant[user]
    pred = predict[user]
    sum_ndsg += ndsg_at_k(relevant=rel, predicted=pred, k=10)
    sum_precision += precision_at_k(relevant=rel, predicted=pred, k=10)
metrics["AVG_nDSG@k"] = sum_ndsg/len(relevant)
metrics["AVG_Precision@k"] = sum_precision/len(relevant)

In [33]:
metrics

{'MAP@k': 0.0072773837301589915,
 'MNAP@k': 0.013485420247542781,
 'Hitrate@k': 0.50501,
 'AVG_nDSG@k': 0.01749793338955469,
 'AVG_Precision@k': 0.07412700000002749}